In [484]:
import pandas as pd
import datetime
import re
import matplotlib.pyplot as plt

In [485]:
data = pd.read_csv('C:\\Users\\RAVID\\Desktop\\Ariel\\3rd year\\semester 2\\כרייה וניתוח נתונים מתקדם בפייתון\\מטלות\\מטלה 2\\matala2_cosmetics_2019-Nov.csv')
data.info()

C:\Users\RAVID\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4635837 entries, 0 to 4635836
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   event_time     object
 1   event_type     object
 2   product_id     int64 
 3   category_id    int64 
 4   category_code  object
 5   brand          object
 6   price          object
 7   user_id        int64 
 8   user_session   object
dtypes: int64(3), object(6)
memory usage: 318.3+ MB


**סעיף 1**

In [486]:
#ננקה את הדאטה מערכים חסרים
dataS = data.sort_values(by=['user_session', 'event_time'])
dataS = dataS[(dataS['user_session'].notnull())]
dataS = dataS.reset_index(drop=True)
dataS

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-05 07:57:05 UTC,view,5829298,1487580011585077370,NaN,farmstay,0.79,556321594,00000dd2-0f5d-4fc9-9d6b-2fc8c7514b04
1,2019-11-11 12:15:10 UTC,view,5771614,2193074740619379535,furniture.living_room.cabinet,kosmekka,150.79,570103701,00001bc6-1e04-4861-be5d-1bc73929c21a
2,2019-11-15 03:50:03 UTC,view,5568472,1487580005595612013,NaN,NaN,3,571677620,00001e77-5db8-4e96-ab7a-022ed3caf8b3
3,2019-11-13 04:37:37 UTC,view,5864850,1487580005268456287,NaN,NaN,12.7,504468592,00004c7d-9507-474e-a559-0fdaccaaa390
4,2019-11-14 03:59:32 UTC,view,5861620,1487580008263189483,NaN,lador,2.25,571211128,0000597b-de39-4a77-9fe5-02c8792ca14e
...,...,...,...,...,...,...,...,...,...
4635019,2019-11-27 09:26:21 UTC,view,5588154,1487580006300255120,NaN,irisk,77.78,577542926,ffff5954-5b7f-4dd8-9fa1-078ae32e684f
4635020,2019-11-27 09:27:35 UTC,view,5676246,1487580006300255120,NaN,marathon,137.78,577542926,ffff5954-5b7f-4dd8-9fa1-078ae32e684f
4635021,2019-11-09 17:02:14 UTC,view,5847328,1982860244379763042,NaN,NaN,8.25,560528468,ffff8044-2a22-4846-8a72-999e870abbe9
4635022,2019-11-02 11:47:34 UTC,view,5528034,1487580005553668971,NaN,NaN,9.52,566764549,ffffbe0a-d2c2-47c7-afab-680bfdfda50d


In [487]:
#נשנה את הטייפ של העמודה כדי שנוכל לבצע בה חיסור
dataS['event_time'] = dataS['event_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
dataS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4635024 entries, 0 to 4635023
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          object        
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 318.3+ MB


In [ ]:
dataS['duration_to_next_event'] = dataS.groupby('user_session')['event_time'].apply(lambda x: x.shift(-1) - x).fillna(pd.Timedelta(seconds=0))
dataS['duration_to_next_event']= dataS['duration_to_next_event'].apply(lambda x: x.total_seconds())
dataS

**סעיף 2**

In [ ]:
dataS['MoreThan'] = dataS.groupby('user_id')['event_time'].apply(lambda x: (x - x.shift(1)) > datetime.timedelta(days=5)).reset_index(drop=True).astype(int)
dataS['funnel_number'] = dataS.groupby('user_id')['MoreThan'].cumsum()
dataS['funnel_number'] = dataS['funnel_number'].fillna(0) + 1
dataS = dataS.drop('MoreThan', axis=1)
dataS

**סעיף 3**

In [ ]:
dataS['one'] = (dataS['user_session'].shift(1) != dataS['user_session']).astype(int)
dataS['index_in_funnel'] = dataS.groupby(['user_id','funnel_number'])['one'].cumsum()
dataS = dataS.drop('one', axis=1)
dataS

**סעיף 4**

In [ ]:
dataS['priceNew']= dataS['price'].apply(lambda x: re.findall('([0-9]+.[0-9]+)', str(x)))
dataS['price'] = dataS['priceNew'].apply(lambda x: float(x[0]))
dataS = dataS.drop('priceNew', axis=1)
dataS.info()

***סעיף 5***

In [ ]:
dataS['event_type'].unique()

In [ ]:
ax = dataS['event_type'].value_counts().plot(kind='bar',
                                    figsize=(8,5), color = 'purple',
                                    title="How many of each type")
ax.set_xlabel("event_type")
ax.set_ylabel("Frequency")
for i, v in enumerate(dataS['event_type'].value_counts()):
    ax.annotate(str(v), xy=(i, v), ha='center', va='bottom')
    
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.show()

**סעיף 6**

In [ ]:
#columns a+b+c+d:
session_data = dataS.groupby('user_session').last()[['user_id', 'funnel_number', 'index_in_funnel']].reset_index()
session_data

In [ ]:
#columns e+f:
session_data = session_data.merge(dataS.groupby('user_session')['event_type'].size().reset_index(name='amount_of_events'), on='user_session')
session_data = session_data.merge(dataS.groupby('user_session')['duration_to_next_event'].sum().reset_index(name='session_duration'), on='user_session')
session_data

In [ ]:
#column g:
view_events = dataS.iloc[np.where(dataS['event_type'] == 'view')][['user_session','product_id']]
session_data = session_data.merge(view_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_viewed'), on='user_session', how='outer')
session_data

In [ ]:
#column h:
cart_events = dataS.iloc[np.where(dataS['event_type'] == 'cart')][['user_session','product_id']]
session_data = session_data.merge(cart_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_added_to_cart'), on='user_session', how='outer')
session_data

In [ ]:
#column i:
purchase_events = dataS.iloc[np.where(dataS['event_type'] == 'purchase')][['user_session','product_id']]
session_data = session_data.merge(purchase_events.groupby('user_session')['product_id'].agg(lambda x: x.tolist()).reset_index(name='list_of_purchased'), on='user_session', how='outer')

In [ ]:
session_data